# **Joins**

In [ ]:
SELECT 2 + 2;

Write a query that displays all the products along with the SalesOrderID even if an order has never been placed for that product. Display OrderId, ProductId, ProductName and only products which weren’t ordered

In [ ]:
SELECT SalesOrderId, p.ProductID, [Name]
FROM Production.Product AS p
LEFT JOIN Sales.SalesOrderDetail AS o
ON p.ProductID = o.ProductID


Write a query to display count each products within order with product name and order date.


In [ ]:
SELECT p.Name, count(p.ProductId) [Ordered], h.OrderDate
FROM Production.Product AS p
JOIN Sales.SalesOrderDetail AS s
ON p.[ProductID] = s.ProductID
JOIN Sales.SalesOrderHeader AS h
ON s.SalesOrderID = h.SalesOrderID
GROUP BY p.Name, h.OrderDate

Write a query to return list of customer names along with a count of the orders made by these customers.


In [ ]:
SELECT p.FirstName, p.LastName, count(c.CustomerID) [Products ordered]
FROM Sales.Customer AS c 
JOIN Person.Person AS p
ON c.PersonID = p.BusinessEntityID
JOIN Sales.SalesOrderHeader AS s
ON c.CustomerID = s.CustomerID
GROUP BY c.CustomerID, p.FirstName, p.LastName

We have been asked by the Products team for some information about every product and its description for those that have a culture originating in English (En). This information is required so that they know what is coming from England: 

 Task1: identification of tables for required information

 Task2: identify common fields in table to link them together. Write the query that join them together and selects the product identifier, culture, product description, culture,, product model name

 Task3: Filtering for the ‘en’ culture. Adding to the query in task2, retrieve inly products that come from English (en) culture.

In [ ]:
SELECT p.ProductID, m.Name [Model Name ], c.Name [Culture], [Description]
FROM Production.Product AS p
JOIN Production.ProductModel As m
ON p.ProductModelID = m.ProductModelID
JOIN Production.ProductModelProductDescriptionCulture AS cid
ON m.ProductModelID = cid.ProductModelID
JOIN Production.Culture AS c 
ON cid.CultureID = c.CultureID
JOIN Production.ProductDescription AS d
ON cid.ProductDescriptionID = d.ProductDescriptionID
WHERE c.Name = 'English'

We have been asked be Adwenture Work Finance Department to find out the total amount due from two customers Walter Brian and Walter Mays on all order they have placed. It is worth noting each time a customer places an order they are treated as a new customer, so they get a separate record.

 Task1: Investigating the Customers’ data for customers who have first name Walter 

 Task2: Identify whis data should be uset to group and join all appropriate tables. 

 Task3: Use the needed aggregate functions

In [51]:
SELECT p.FirstName, p.LastName, SUM(h.TotalDue) [Total Due]
FROM Sales.Customer AS s
JOIN Person.Person AS p
ON s.PersonID = p.BusinessEntityID
JOIN Sales.SalesOrderHeader AS h
ON s.CustomerID = h.CustomerID
WHERE p.FirstName like 'Walter' and p.LastName in ('Brian', 'Mays')
GROUP BY p.LastName, p.FirstName

(2 rows affected)

Total execution time: 00:00:00.086

FirstName,LastName,Total Due
Walter,Brian,6504.8271
Walter,Mays,51015.5565


We want to see which employee have been working in more than one department. Use the tables Person. Person, HumanResources.Employee and HumanResources.EmployeeDepartmentHistory. The COUNT function in companion with GROUP BY and HAVING is used to find out the answer to our question.

In [59]:
SELECT p.FirstName, p.LastName, COUNT(DepartmentID) [N of departments]
FROM HumanResources.Employee AS e
JOIN HumanResources.EmployeeDepartmentHistory AS h
ON e.BusinessEntityID = h.BusinessEntityID
JOIN Person.Person AS p 
ON e.BusinessEntityID = p.BusinessEntityID
GROUP BY e.BusinessEntityID, p.FirstName, p.LastName
HAVING COUNT(DepartmentID) > 1


(5 rows affected)

Total execution time: 00:00:00.061

FirstName,LastName,N of departments
Rob,Walters,2
David,Bradley,2
William,Vong,2
Laura,Norman,2
Sheela,Word,3


We want to see information about countries and provinces. Join the CountryRegion and the StateProvince tables (both in the Person schema). Note that we want to keep the countries for which there are no provinces! Sort the result on country name and province name.

In [62]:
SELECT r.Name [Country Name], p.Name [Province Name]
FROM Person.CountryRegion AS r
JOIN Person.StateProvince AS p
ON r.CountryRegionCode = p.CountryRegionCode
WHERE p.IsOnlyStateProvinceFlag = 1
ORDER BY r.Name, p.Name

(8 rows affected)

Total execution time: 00:00:00.098

Country Name,Province Name
American Samoa,American Samoa
France,France
Marshall Islands,Marshall Islands
Micronesia,Micronesia
Northern Mariana Islands,Northern Mariana Islands
Palau,Palau
United Kingdom,England
"Virgin Islands, U.S.",Virgin Islands


# **Subqueries**